# My hangman file

In [1]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections

In [4]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


In [37]:
random.choice(full_dict)

'photogravurist'

So, I think the best way to do this would be to make a decision tree from scratch, but I think there is an issue of outliers that I am not super sure how to deal with. I think the dictionary is big enough that there are some words that the algorithm should figure out how to throw out. Like if there are 2 moves left, it should make the highest probability assumption it can which eliminates trees with more than 2 moves of surprise.

So... I think how the algorithm should work is, it doesnt just pick the ONE letter that would maximize the entropy next, it should find like a PATH of guesses that maximally reduces the entropy.

I am assuming there are always 6 tries, which seems a bit low...

It seems a bit complex though... If I build a tree I don't have to constantly re-calculate it, but I just move from node to node.

HOW DO I MAKE A DECISION TREE CLASSIFIER WITH NO LABELS THOUGH?

For testing purposes, I want to just remake hangman here so I can mess with it. So gut all of the connection to some external anything. I think I know how it works from running it a bit.

In [38]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)


    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        clean_word = word.replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                print("Success! the word was: %s" % word)
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            print("You Lose, the answer was: %s" % self.correct_word)
                


            



        """""           
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
    """""
        

In [39]:
word = "_____"
correctWord = "apple"
letter = "z"
#I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
indices = [i for i, x in enumerate(list(correctWord)) if x == letter]
wordList = list(word)
for i in indices:
    wordList[i] = letter #now I replace the occurrences of

word = "".join(wordList)

#while lidx != -1:
#    word[lidx] = letter
#    lidx = correctWord.find(letter)
print(word)

_____


In [40]:
game = MyHangman()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: ______________.
aaa
Guessing letter: e
Got a Letter, e, # of tries remaining: 6. Word: _____________e.
abdominocystic
Guessing letter: a
Got a Letter, a, # of tries remaining: 6. Word: __a____a_____e.
acclimatisable
Guessing letter: n
Got a Letter, n, # of tries remaining: 6. Word: __an___an___ne.
cyanoguanidine
Guessing letter: i
Got a Letter, i, # of tries remaining: 6. Word: __an___ani_ine.
cyanoguanidine
Guessing letter: c
Got a Letter, c, # of tries remaining: 6. Word: c_an___ani_ine.
cyanoguanidine
Guessing letter: y
Got a Letter, y, # of tries remaining: 6. Word: cyan___ani_ine.
cyanoguanidine
Guessing letter: o
Got a Letter, o, # of tries remaining: 6. Word: cyano__ani_ine.
cyanoguanidine
Guessing letter: g
Got a Letter, g, # of tries remaining: 6. Word: cyanog_ani_ine.
cyanoguanidine
Guessing letter: u
Got a Letter, u, # of tries remaining: 6. Word: cyanoguani_ine.
cyanoguanidine
Guessing letter: d
Success! the word

Ugh, after looking at this, I need to modify the MyHangman code to get some data from it right?

Ok... finally I can start writing the strategy... that shouldn't be too difficult right (uhhh)



My understanding is that there are 2 ways to do this... one that is the theoretical best strategy and the other that is just a good heuristic.

https://cs.stackexchange.com/questions/128667/what-is-the-optimal-algorithm-for-playing-the-hangman-word-game

The good one is to look at the game like a decision tree, see which choices have the highest entropy and make those. Of course there would be complications like you plan your route through the tree, and you have to figure out which nodes should be inaccessible?

The other is to get a game state function and use it to construct a matrix and then minimize it? I don't really get it right now. This uses dynamic programming, which is optimization over plain recursion?

I guess so far I have assumed that there is an equal probability of each word being chosen, which may be untrue.

So... let's say the word is $apple$, and look at a bit...

first prune dict to 5 letter words

In [41]:
corr_word = "apple"
word = "_____"
guessed_letters = []
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dict)).most_common()


###############################################
# Replace with your own "guess" function here #
###############################################

# clean the word so that we strip away the space characters
# replace "_" with "." as "." indicates any character in regular expressions
clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
clean_word = word.replace("_",".")
print(word)
#print(word[::2])
print(clean_word)

# find length of passed word
len_word = len(clean_word)

# grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
current_dictionary = full_dict

new_dictionary = []

# iterate through all of the words in the old plausible dictionary
for dict_word in current_dictionary:
    # continue if the word is not of the appropriate length
    if len(dict_word) != len_word:
        continue
        
    # if dictionary word is a possible match then add it to the current dictionary
    if re.match(clean_word,dict_word):
        new_dictionary.append(dict_word)

# overwrite old possible words dictionary with updated version
current_dictionary = new_dictionary


# count occurrence of all characters in possible word matches
full_dict_string = "".join(new_dictionary)

c = collections.Counter(full_dict_string)
sorted_letter_count = c.most_common()                   

guess_letter = '!'

# return most frequently occurring letter in all possible words that hasn't been guessed yet
for letter,instance_count in sorted_letter_count:
    if letter not in guessed_letters:
        guess_letter = letter
        break
    
# if no word matches in training dictionary, default back to ordering of full dictionary
if guess_letter == '!':
    sorted_letter_count = full_dictionary_common_letter_sorted
    for letter,instance_count in sorted_letter_count:
        if letter not in guessed_letters:
            guess_letter = letter
            break            

#return guess_letter
print(guess_letter)


_____
.....
a


In [42]:
corr_word = "apple"
word = "_____"
guessed_letters = []
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dict)).most_common()
all_letters = [x[0] for x in full_dictionary_common_letter_sorted]


###############################################
# Replace with your own "guess" function here #
###############################################

# clean the word so that we strip away the space characters
# replace "_" with "." as "." indicates any character in regular expressions
clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
clean_word = word.replace("_",".")
#print(word)
#print(word[::2])
#print(clean_word)

# find length of passed word
len_word = len(clean_word)

# grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
current_dictionary = full_dict

new_dictionary = []

# iterate through all of the words in the old plausible dictionary
for dict_word in current_dictionary:
    # continue if the word is not of the appropriate length
    if len(dict_word) != len_word:
        continue
        
    #NOTE THIS MAY NOT BE HOW I DO THIS AFTER I DO THE STUFF BELOW!

    # if dictionary word is a possible match then add it to the current dictionary
    if re.match(clean_word,dict_word):
        new_dictionary.append(dict_word)

# overwrite old possible words dictionary with updated version
current_dictionary = new_dictionary


# count occurrence of all characters in possible word matches
full_dict_string = "".join(new_dictionary)

c = collections.Counter(full_dict_string)
sorted_letter_count = c.most_common()                   

guess_letter = '!'


# what options do we still have to guess?
letter_options = [letter for letter in all_letters if letter not in guessed_letters]

# break down the possibilities by letter if that letter is correct
future_guess_array = []
guess_entropy_array = []

def calc_entropy(base, subset): 
    # This calculates the entropy of taking a set A and breaking it down into a subset B and B complement
    # I mean mathematically, this acts on lists rather sets programming wise.
    if base == subset: #hmmmmmm how do I handle this?
        return 0
    else:
        p = len(subset) / len(base)
        return -(p*math.log(p) + (1-p)*math.log(1-p))
        
# for each letter, calculate the entropy of choosing that letter
for letter in letter_options:
    dict_by_letter = []

    for word in current_dictionary:
        if set(letter) & set(word): # I use the sets to get all the words with a specific letter
            dict_by_letter.append(word)
    future_guess_array.append(dict_by_letter)
    guess_entropy = calc_entropy(current_dictionary, dict_by_letter)
    guess_entropy_array.append(guess_entropy)

# now, which letter had the highest entropy?
idx_max = guess_entropy_array.index(max(guess_entropy_array))
best_letter = letter_options[idx_max]
guess_letter = best_letter





#return guess_letter
print(guess_letter)


a


In [43]:
# The structure of a node has to contain what data:
# The guessed letters, including whatever one got to this node, root would be empty
# The dictionary after guessing a letter
# 

For this tree though, guessing A then B would get us to a different node than B then A, but these should take us to the same state... does it make sense to have it work like this? I guess game trees work like this?

In [2]:
class MyHangman2(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.has_won = False
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)

    def get_winstate(self):
        return self.has_won


    def guess(self, word): # word input example: "_ p p _ e "
        all_letters = [x[0] for x in self.full_dictionary_common_letter_sorted]


        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        clean_word = word.replace("_",".")
        #print(word)
        #print(word[::2])
        #print(clean_word)

        # find length of passed word
        len_word = len(clean_word)

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary

        new_dictionary = []

        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            #NOTE THIS MAY NOT BE HOW I DO THIS AFTER I DO THE STUFF BELOW!

            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)

        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary


        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)                 

        guess_letter = '!'


        # what options do we still have to guess?
        letter_options = [letter for letter in all_letters if letter not in self.guessed_letters]

        # break down the possibilities by letter if that letter is correct
        # future_guess_array = []
        # guess_entropy_array = []

        def calc_entropy(base, subset): 
            # This calculates the entropy of taking a set A and breaking it down into a subset B and B complement
            # I mean mathematically, this acts on lists rather sets programming wise.

            #I have domain issues if p=1 or p=0, so if the base == subset or if subset empty
            if base == subset: #hmmmmmm how do I handle this?
                return 1
            elif len(subset) == 0:
                return 0
            else:
                p = len(subset) / len(base)
                return -(p*math.log(p) + (1-p)*math.log(1-p))
                
        # for each letter, calculate the entropy of choosing that letter
        word_count_total = len(current_dictionary)
        guess_entropy_array = []

        for letter in letter_options:
            #dict_by_letter = []
            word_count_by_letter = 0

            # This process here is very slow, for each letter it counts how many words in the dictionary have that letter present.
            # I think there is a better way to do this...
            for word in current_dictionary:
                if set(letter) & set(word): # I use sets to get all the words with a specific letter
                    #dict_by_letter.append(word)
                    word_count_by_letter+=1
            
            
            p = word_count_by_letter / word_count_total


            # p=0 and p=1 are special and need to treat them differently, they are out of the domain of log
            # p=0 means there are no words remaining in the dictionary, so that guess is useless and I set the entropy to 0
            # p=1 will only occur when the number of words corresponding to a letter is the same as in the entire dictionary
            # this will only occur when we have locked in on one possible answer, so I set the entropy to 1 which is greater than the normal max the entropy can have

            if p == 0:
                letter_entropy = 0
            elif p == 1:
                letter_entropy = 1
            else:
                letter_entropy = -(p*math.log(p) + (1-p)*math.log(1-p))
            #future_guess_array.append(dict_by_letter)
            #guess_entropy = calc_entropy(current_dictionary, dict_by_letter)
            #guess_entropy_array.append(guess_entropy)
            guess_entropy_array.append(letter_entropy)

        # now, which letter had the highest entropy?
        idx_max = guess_entropy_array.index(max(guess_entropy_array))
        best_letter = letter_options[idx_max]
        guess_letter = best_letter
                    
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        self.has_won = False
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                


            



        """""           
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
    """""

In [12]:
game = MyHangman2()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _________________.
Guessing letter: n
Got a Letter, n, # of tries remaining: 6. Word: ________n________.
Guessing letter: p
Incorrect guess p, # of tries remaining: 6. Word: ________n________.
Guessing letter: c
Got a Letter, c, # of tries remaining: 5. Word: ___c__c_n________.
Guessing letter: l
Got a Letter, l, # of tries remaining: 5. Word: ___c__c_n__l_____.
Guessing letter: e
Got a Letter, e, # of tries remaining: 5. Word: ___ce_c_n__le__e_.
Guessing letter: i
Got a Letter, i, # of tries remaining: 5. Word: __ice_c_n__le__e_.
Guessing letter: a
Got a Letter, a, # of tries remaining: 5. Word: __iceac_n__le__e_.
Guessing letter: o
Got a Letter, o, # of tries remaining: 5. Word: __iceac_no_le__e_.
Guessing letter: t
Got a Letter, t, # of tries remaining: 5. Word: t_iceac_no_le__e_.
Guessing letter: d
Got a Letter, d, # of tries remaining: 5. Word: t_iceac_no_led_ed.
Guessing letter: g
Got a Letter, g, # of tries remaining:

In [47]:
game.get_winstate()

True

In [3]:
N = 100
wincount = 0
game = MyHangman2()
t = time.time()

for i in range(N):
    elapsed = time.time() - t
    if (i+1) % 10 == 0:
        print("On Game %i, elapsed time is %f, current winrate is %f" % (i+1, elapsed ,100*wincount/(i)))
    game.start_game(verbose = False)
    wincount += game.get_winstate()
    #if(game.get_winstate()):
    #    print("Won game %i, win count = %i, time elapsed = %f" % (i+1, wincount, elapsed))
print("Final Score: %f percent wins. Time elapsed: %f" % (100 * wincount / N, elapsed))

On Game 10, elapsed time is 33.335229, current winrate is 77.777778
On Game 20, elapsed time is 71.464617, current winrate is 89.473684
On Game 30, elapsed time is 111.827947, current winrate is 82.758621
On Game 40, elapsed time is 148.977046, current winrate is 84.615385
On Game 50, elapsed time is 191.582550, current winrate is 81.632653
On Game 60, elapsed time is 229.741698, current winrate is 79.661017
On Game 70, elapsed time is 267.933931, current winrate is 81.159420
On Game 80, elapsed time is 307.056595, current winrate is 79.746835
On Game 90, elapsed time is 348.330910, current winrate is 79.775281
On Game 100, elapsed time is 388.220368, current winrate is 80.808081
Final Score: 81.000000 percent wins. Time elapsed: 388.220368


with word_count method, takes 8.2 secs for 10 games... 18 sec for 25 games

With the large lists, takes 9.0 secs for 10 games... 27 sec for 25 games

In [5]:
class MyHangman3(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.has_won = False
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)

    def get_winstate(self):
        return self.has_won


    def guess(self, word): # word input example: "_ p p _ e "
        all_letters = [x[0] for x in self.full_dictionary_common_letter_sorted]


        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        clean_word = word.replace("_",".")
        #print(word)
        #print(word[::2])
        #print(clean_word)

        # find length of passed word
        len_word = len(clean_word)

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []

        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                

            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)

        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary

        guess_letter = '!'


        # what options do we still have to guess?
        letter_options = [letter for letter in all_letters if letter not in self.guessed_letters]

        # break down the possibilities by letter if that letter is correct
        #future_guess_array = []
        guess_entropy_array = []

        def calc_entropy(base, subset): 
            # This calculates the entropy of taking a set A and breaking it down into a subset B and B complement
            # I mean mathematically, this acts on lists rather sets programming wise.

            #I have domain issues if p=1 or p=0, so if the base == subset or if subset empty
            if base == subset: #hmmmmmm how do I handle this?
                return 1
            elif len(subset) == 0:
                return 0
            else:
                p = len(subset) / len(base)
                return -(p*math.log(p) + (1-p)*math.log(1-p))
                
        # for each letter, calculate the entropy of choosing that letter
        #word_count_total = len(current_dictionary)

        for letter in letter_options:
            dict_by_letter = []
            #word_count_by_letter = 0

            # This process here is very slow, for each letter it counts how many words in the dictionary have that letter present.
            # I think there is a better way to do this...
            for word in current_dictionary:
                if set(letter) & set(word): # I use sets to get all the words with a specific letter
                    dict_by_letter.append(word)
                    #word_count_by_letter+=1
            
            




            
            #p = word_count_by_letter / word_count_total


            # p=0 and p=1 are special and need to treat them differently, they are out of the domain of log
            # p=0 means there are no words remaining in the dictionary, so that guess is useless and I set the entropy to 0
            # p=1 will only occur when the number of words corresponding to a letter is the same as in the entire dictionary
            # this will only occur when we have locked in on one possible answer, so I set the entropy to 1 which is greater than the normal max the entropy can have

            #if p == 0:
            #    letter_entropy = 0
            #elif p == 1:
            #    letter_entropy = 1
            #else:
            #    letter_entropy = -(p*math.log(p) + (1-p)*math.log(1-p))
            #future_guess_array.append(dict_by_letter)
            guess_entropy = calc_entropy(current_dictionary, dict_by_letter)
            guess_entropy_array.append(guess_entropy)
            #guess_entropy_array.append(letter_entropy)

        # now, which letter had the highest entropy?
        idx_max = guess_entropy_array.index(max(guess_entropy_array))
        best_letter = letter_options[idx_max]
        guess_letter = best_letter
                    
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        self.has_won = False
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                

In [14]:
game = MyHangman3()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: ________.
Guessing letter: n
Got a Letter, n, # of tries remaining: 6. Word: ______n_.
Guessing letter: r
Incorrect guess r, # of tries remaining: 6. Word: ______n_.
Guessing letter: e
Incorrect guess e, # of tries remaining: 5. Word: ______n_.
Guessing letter: g
Incorrect guess g, # of tries remaining: 4. Word: ______n_.
Guessing letter: a
Got a Letter, a, # of tries remaining: 3. Word: _____an_.
Guessing letter: o
Incorrect guess o, # of tries remaining: 3. Word: _____an_.
Guessing letter: t
Got a Letter, t, # of tries remaining: 2. Word: ___t_an_.
Guessing letter: i
Incorrect guess i, # of tries remaining: 2. Word: ___t_an_.
Guessing letter: s
Got a Letter, s, # of tries remaining: 1. Word: __st_ans.
Guessing letter: l
Incorrect guess l, # of tries remaining: 1. Word: __st_ans.
You Lose, the answer was: dustpans


In [10]:
for i in range(10):
    game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: n
Incorrect guess n, # of tries remaining: 6. Word: _________.
Guessing letter: s
Incorrect guess s, # of tries remaining: 5. Word: _________.
Guessing letter: o
Got a Letter, o, # of tries remaining: 4. Word: _o_______.
Guessing letter: t
Got a Letter, t, # of tries remaining: 4. Word: to_______.
Guessing letter: r
Got a Letter, r, # of tries remaining: 4. Word: torr_____.
Guessing letter: i
Got a Letter, i, # of tries remaining: 4. Word: torr__i__.
Guessing letter: e
Got a Letter, e, # of tries remaining: 4. Word: torre_ie_.
Guessing letter: d
Got a Letter, d, # of tries remaining: 4. Word: torre_ied.
Guessing letter: f
Success! the word was: torrefied
Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: n
Incorrect guess n, # of tries remaining: 6. Word: _________.
Guessing letter: s
Got a Letter, s, # of tries remaining: 5. Word: ______s_s.
Guessing letter:

In [6]:
N = 100
wincount = 0
game = MyHangman3()
t = time.time()

for i in range(N):
    elapsed = time.time() - t
    if (i+1) % 10 == 0:
        print("On Game %i, elapsed time is %f, current winrate is %f" % (i+1, elapsed ,100*wincount/(i)))
    game.start_game(verbose = False)
    wincount += game.get_winstate()
    #if(game.get_winstate()):
    #    print("Won game %i, win count = %i, time elapsed = %f" % (i+1, wincount, elapsed))
print("Final Score: %f percent wins. Time elapsed: %f" % (100 * wincount / N, elapsed))

On Game 10, elapsed time is 35.683603, current winrate is 88.888889
On Game 20, elapsed time is 74.881002, current winrate is 84.210526
On Game 30, elapsed time is 114.153351, current winrate is 82.758621
On Game 40, elapsed time is 151.242779, current winrate is 87.179487
On Game 50, elapsed time is 191.093898, current winrate is 85.714286
On Game 60, elapsed time is 232.500932, current winrate is 88.135593
On Game 70, elapsed time is 272.096708, current winrate is 86.956522
On Game 80, elapsed time is 311.778911, current winrate is 86.075949
On Game 90, elapsed time is 349.470736, current winrate is 86.516854
On Game 100, elapsed time is 389.532912, current winrate is 86.868687
Final Score: 87.000000 percent wins. Time elapsed: 389.532912


Hmmmmmm not really sure why... but the subset focused one has a higher winrate than the wordcount one, and theyre about the same speed...